# define FASTAReader function 

In [1]:
def FASTAReader(file):
    line = file.readline()
    assert line.startswith('>'), "Not a FASTA file"
    seq_id = line[1:].rstrip('\r\n')
    sequence = []
    line = file.readline()
    sequences = []
    while line:
        if line.startswith('>'):
            sequences.append((seq_id, ''.join(sequence)))
            seq_id = line[1:].rstrip('\r\n')
            sequence = []
        else:
            sequence.append(line.strip())
        
        line = file.readline()
    sequences.append((seq_id, ''.join(sequence)))
    return sequences

# open files 

In [2]:
#open files with FASTAReader
AA_seq= FASTAReader(open('mafft_aligned'))
nuc_seq= FASTAReader(open('seqdump (1).txt'))
query_seq= FASTAReader(open('week4_query.fa'))

# make dictionary

In [3]:
#make dictionary
dict={}
#create a loop to add nucleotides and AA's to the dictionary with the header (seqID) as the key
for seq_id, sequence in nuc_seq:
    key1=seq_id.split()
    key2=key1[0]
    dict[key2]=[sequence]
for seq_id, sequence in AA_seq:
    key3=seq_id.split()
    key4=key3[0]
    key5=key4[:-2]
    dict[key5].append(sequence)

# add gaps to nucleotides 

In [4]:
#define variables in order to add new values to the dictionary 
newDNA_list=[]
counter=0
for key, value in dict.items():
    counter += 1
    old_DNA= value[0]
    amino_acids= value[1]
    new_DNA= ""
#look into the amino acids
    #if the character is a space, add three spaces to the new DNA column 
    #if the character is not a space, slice the string and add three nucleotides to the column in dictionary 
    for j, k in enumerate(amino_acids):
        if k == "-":
            new_DNA += "---"
        else:
            new_DNA += old_DNA[j:(j+3)]
    value.append(new_DNA)
    newDNA_list.append(new_DNA)
    #if counter >1:
        #break
    
counter2=0
for key, value in dict.items():
    counter2 += 1
    #print(key, value)
    
    if counter2 > 1: 
        break 

# split query into codons 

In [5]:
query_codons=[]
query_list=[]
for seq_id, sequence in query_seq:
    key1=seq_id.split()
    key2=key1[0]
    query_list.append(sequence)

string= query_list[0]
n = 3
query_codons = [(string[i:i+n]) for i in range(0, len(string), 3)] 
#print(query_codons)

# split BLAST sequences into codons

In [10]:
blast_codons=[]
for character in newDNA_list:
    string=character
    n = 3
    codons= [(string[i:i+n]) for i in range(0, len(string), 3)]
    blast_codons.append(codons)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# get number of  synonymous and non-synonymous mutations 

In [7]:
codontable = { 
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                  
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
    'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_', 
    'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W', 
} 

In [30]:
def get_val(codon): 
    
    for key, value in codontable.items(): 
            if codon == key: 
                return value 
    return "key not in dictionary"


6008
6008


In [153]:
synonymous_counter=0
nonsynonymous_counter=0
ds_dN={}
values=[]

for i in blast_codons:
    for b in i:
        for a in blast_codons[0]:
            if b == "---" or a == "---":
                pass 
            if a == b: 
                pass 
            else:
                ds_dN.setdefault(a, [])
                
                a_value=get_val(a)
                b_value=get_val(b)

                if a_value == b_value:
                    
                    synonymous_counter += 1
                    ds_dN[a].append("SYN")
                    if synonymous_counter > 6008: 
                        break 
                
                else:
                    nonsynonymous_counter += 1
                    ds_dN[a].append("NON")
                    if nonsynonymous_counter > 6008: 
                        break 
                        

In [165]:
if "SYN" in ds_dN.values():
    print("TRUE")
else: 
    print("FALSE")

FALSE


In [166]:
print(synonymous_counter)
dNdS= synonymous_counter/nonsynonymous_counter
print(dNdS)

7445
0.0012382392221629021


In [ ]:
import pandas as pd
from scipy.stats import stats 
